# Skript to infer a wohle WSI with a pretrained FasterRCNN

    1. Load list of slides to infer
    2. Load pretrained FasterRCNN 
    3. Inference loop

## 1. Load list of slides to infer

In [3]:
%load_ext autoreload
%autoreload 2

import pickle
import MaPSlideDetectionInferenzLoader2
import openslide
import os 

# WSIs to infer
wsi_dir = "wsis/meningioma_grading_training_dataset"

files_for_inference = {
    "Meningeome_1000":list()
}

for file in os.listdir(wsi_dir):
    if file.endswith(".ndpi"):
        files_for_inference['Meningeome_1000'].append(file)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
print(f"Number of files to infer: {len(files_for_inference['Meningeome_1000'])}")

Number of files to infer: 998


## 2. Load pretrained FasterRCNN

In [3]:
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator

# load a model pre-trained pre-trained on COCO
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
num_classes = 2  
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
resnet18 = torchvision.models.resnet18(pretrained = True)
backbone = nn.Sequential(*(list(resnet18.children())[:-2]))
backbone.out_channels = 512
# Anchor sizes were determined heuristically
anchor_generator = AnchorGenerator(sizes=((307,358,409,460),),aspect_ratios=((1.0),))
roi_pooler = torchvision.ops.MultiScaleRoIAlign(featmap_names=['0'],
                                                output_size=7,
                                                sampling_ratio=2)

# Flange different parts together to form a FasterRCNN model 
model = FasterRCNN(backbone,num_classes=num_classes,
                   rpn_anchor_generator=anchor_generator,
                   box_roi_pool=roi_pooler)

In [4]:
# Load Model for inference
model_file = "FasterRCNN.pth"

model.load_state_dict(torch.load(model_file))

<All keys matched successfully>

In [5]:
# bring model to gpu
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
    #device = 'cpu'
model.to(device)

FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
       

## 3. Inference Loop

In [6]:
import torchvision
import progressbar
import math

def predict_on_wsi(test_loader,model):
    '''predict on a whole WSI'''
    boxes = list()
    scores = list()

    pbar = progressbar.ProgressBar(max_value = len(test_loader))

    with torch.no_grad():
        if model.training:
            model.eval()
        for idx, (images,targets) in enumerate(test_loader):
            images = [image.to(device) for image in images]
            predictions = model(images)

            for pred,target in zip(predictions,targets):
                if len(pred['boxes'])!=0:
                    for box,score in zip(pred['boxes'],pred['scores']):
                        box = box.to('cpu')
                        # Project boxes back onto WSI
                        box[0], box[2] = box[0] + target['coords'][0], box[2] + target['coords'][0]
                        box[1], box[3] = box[1] + target['coords'][1], box[3] + target['coords'][1]

                        boxes.append(box)
                        scores.append(score.to('cpu'))
            pbar.update(idx)
            
    size = len(boxes)
    # Non maximum surpression because the patches of the wsi will have an overlap, so there might be doubled detections
    final_boxes = torch.cat(boxes).reshape(size,4)
    final_scores = torch.stack(scores)

    keep = torchvision.ops.nms(final_boxes,final_scores,iou_threshold = 0.2)
    final_boxes = final_boxes[keep]
    final_scores = final_scores[keep]
    return(final_boxes,final_scores)


In [9]:
import os
import shutil
from torchvision import transforms
from torch.utils.data import DataLoader
import pickle
from pathlib import Path
from openslide import OpenSlideError
from PIL.Image import UnidentifiedImageError

transform = transforms.Compose(
    [transforms.ToTensor()])

batch_size = 8
# Dir where to save inferrence results
save_path = "Inferences/"
# File name for inferrence results
file_name = "detection_results_"


case_list = ['Meningeome_1000']

# Dir where WSIs are stored
source_dir = "wsis/meningioma_grading_training_dataset"

for case  in case_list:
    
    for file in files_for_inference[case]:
        if os.path.exists(save_path + file.split(' ')[0] + ".p"):
            print(f"File {file} already inferred. Going on with next file.")
            continue
            
        slide_file = source_dir + '/' + case + '/' + file
        # do inferrecne
        
        try:

            slide = openslide.open_slide(slide_file)
            # Initialise lodaer for inference
            test_data = MaPSlideDetectionInferenzLoader2.MyMapSlideDetectionDataset(slide, transforms = transform, width= 1024, height= 1024)
            test_loader = DataLoader(test_data, batch_size = batch_size, collate_fn=test_data.collate_fn, num_workers=8)
            # Infere on WSI
            print(f"Inferring file {file}")
            pred_boxes,pred_scores = predict_on_wsi(test_loader,model)
            
            # Safe result
            detection_results = {
            'boxes':pred_boxes,
            'scores':pred_scores,
            'file':file

            }
            pickle.dump(detection_results, open(save_path + file.split(' ')[0] + ".p","wb"))
        except FileNotFoundError:
            print(f"File {file} not found, trying next one.")
        except OpenSlideError:
            print(f"File {file} corrupted. Can't validate JPEG for directory 0: Expected marker at 4294972160, found none")
        except UnidentifiedImageError:
            print(f"Unidentified Image Error in Image {file}, going on with next file")
            
print(f"done")

File N1015_09_III2_HE - 2019-11-22 10.15.48.ndpi already inferred. Going on with next file.
File N856_10_1_HE - 2019-11-29 20.22.31.ndpi already inferred. Going on with next file.
File N1037_09_I2_HE - 2019-11-22 11.32.00.ndpi already inferred. Going on with next file.
File N1549_11_II_HE - 2019-12-12 20.57.08.ndpi already inferred. Going on with next file.
File N1169_10_2_HE - 2019-12-02 19.39.36.ndpi already inferred. Going on with next file.
File N156_09_I3_HE - 2019-11-19 20.56.13.ndpi already inferred. Going on with next file.
File N1193_09_1_HE - 2019-11-22 20.42.01.ndpi already inferred. Going on with next file.
File N411_10_3_HE - 2019-11-28 10.50.25.ndpi already inferred. Going on with next file.
File N469_11_3_HE - 2019-12-06 11.19.47.ndpi already inferred. Going on with next file.
File N829_11_I_HE - 2019-12-10 10.48.45.ndpi already inferred. Going on with next file.
File N156_09_I1_HE - 2019-11-19 20.23.15.ndpi already inferred. Going on with next file.
File N939_11_4_HE - 

/usr/local/lib/python3.8/dist-packages/PIL/TiffImagePlugin.py:793: UserWarning: Corrupt EXIF data.  Expecting to read 2 bytes but only got 0. 
  warnings.warn(str(msg))


Inferring file N522_11_HE - 2019-12-09 11.21.00.ndpi


 99% (663 of 665) |##################### | Elapsed Time: 0:01:27 ETA:   0:00:00

Inferring file N254_10_II_HE - 2019-11-27 18.00.30.ndpi


...

Inferring file N1266_10_I2_HE - 2019-12-03 13.35.27.ndpi


 99% (926 of 928) |##################### | Elapsed Time: 0:02:20 ETA:   0:00:00

Inferring file N1382_10_1_HE - 2019-12-03 17.41.34.ndpi


 46% (687 of 1482) |#########            | Elapsed Time: 0:01:52 ETA:   0:02:01

KeyboardInterrupt: 